### Домашняя работа №3

1. Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import r2_score
from sklearn import datasets
from sklearn.model_selection import train_test_split
from torch import optim
import warnings
import math
warnings.filterwarnings('ignore')

In [2]:
class CaliforniaDataset(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, init_target, transform=None):
        self._base_dataset = torch.from_numpy(init_dataset).type(torch.float)
        self._base_targets = torch.from_numpy(init_target).type(torch.float)
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        features = self._base_dataset[idx]
        target = self._base_targets[idx]

        if self.transform is not None:
            features = self.transform(features)
      
        return features, target

In [3]:

class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation == "relu":
            return F.relu(x)
        if self.activation == "sigmoid":
            return F.sigmoid(x)
        if self.activation == "leaky_relu":
            return F.leaky_relu(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim, 'leaky_relu')
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.15)
        self.fc2 = Perceptron(hidden_dim, 1, 'leaky_relu')
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)

        return x
     

In [4]:
data = datasets.fetch_california_housing()
     

X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size = 0.25, random_state = 13)
     

train_dataset = CaliforniaDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=150, shuffle=False)
     

test_dataset = CaliforniaDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False)
     

net = FeedForward(8, 8)

In [5]:
def train_nn(opt='adam', num_epochs = 20):
    
    criterion = nn.MSELoss()

    if opt == 'adam':
            optimizer = optim.Adam(net.parameters(), lr=0.001)
    elif opt == 'rmsprop':
            optimizer = optim.RMSprop(net.parameters(), lr=0.001)
    elif opt == 'sgd':
            optimizer = optim.SGD(net.parameters(), lr=0.001)
    for epoch in range(num_epochs):  
        running_loss, running_items, r2 = 0.0, 0.0, 0.0

        for i, data in enumerate(train_loader):
            fets, target = data[0], data[1]

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(fets)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_items += len(target)

            predict = outputs.data.numpy()
            tr_target = target.view(target.shape[0], 1).numpy()
            r2 += r2_score(tr_target, predict)

            if i % 30 == 0:
                net.eval()

                data = list(test_loader)[0]

                test_outputs = net(data[0])
                test_predict = test_outputs.data.numpy()
                te_target = data[1].view(data[1].shape[0], 1)
                test_r2 = r2_score(te_target, test_predict)

                print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                      f'Step [{i + 1}/{len(train_loader)}]. ' \
                      f'Loss: {running_loss / running_items:.3f}. ' \
                      f'r2: {r2:.3f}. ' \
                      f'Test r2: {test_r2:.3f}')

                running_loss, running_items, r2 = 0.0, 0.0, 0.0

                net.train()

    print('Training is finished!')

In [6]:
train_nn(opt='adam', num_epochs=10)

Epoch [1/10]. Step [1/104]. Loss: 0.041. r2: -3.086. Test r2: -5.301
Epoch [1/10]. Step [31/104]. Loss: 0.036. r2: -98.596. Test r2: -5.263
Epoch [1/10]. Step [61/104]. Loss: 0.036. r2: -93.125. Test r2: -5.072
Epoch [1/10]. Step [91/104]. Loss: 0.035. r2: -90.287. Test r2: -4.236
Epoch [2/10]. Step [1/104]. Loss: 0.036. r2: -2.640. Test r2: -3.703
Epoch [2/10]. Step [31/104]. Loss: 0.031. r2: -82.019. Test r2: -3.781
Epoch [2/10]. Step [61/104]. Loss: 0.029. r2: -69.174. Test r2: -4.191
Epoch [2/10]. Step [91/104]. Loss: 0.024. r2: -53.114. Test r2: -2.858
Epoch [3/10]. Step [1/104]. Loss: 0.022. r2: -1.136. Test r2: -2.415
Epoch [3/10]. Step [31/104]. Loss: 0.019. r2: -35.312. Test r2: -1.362
Epoch [3/10]. Step [61/104]. Loss: 0.017. r2: -25.928. Test r2: -0.836
Epoch [3/10]. Step [91/104]. Loss: 0.016. r2: -23.776. Test r2: -0.372
Epoch [4/10]. Step [1/104]. Loss: 0.016. r2: -0.499. Test r2: -0.884
Epoch [4/10]. Step [31/104]. Loss: 0.015. r2: -21.472. Test r2: -0.418
Epoch [4/10]. 

In [7]:
train_nn(opt='rmsprop', num_epochs=10)

Epoch [1/10]. Step [1/104]. Loss: 0.011. r2: -0.075. Test r2: -1.002
Epoch [1/10]. Step [31/104]. Loss: 0.010. r2: -3.307. Test r2: -0.207
Epoch [1/10]. Step [61/104]. Loss: 0.010. r2: -2.832. Test r2: -0.133
Epoch [1/10]. Step [91/104]. Loss: 0.009. r2: -2.611. Test r2: -0.162
Epoch [2/10]. Step [1/104]. Loss: 0.011. r2: -0.096. Test r2: -0.242
Epoch [2/10]. Step [31/104]. Loss: 0.009. r2: -2.156. Test r2: -0.128
Epoch [2/10]. Step [61/104]. Loss: 0.009. r2: -2.263. Test r2: -0.163
Epoch [2/10]. Step [91/104]. Loss: 0.009. r2: -2.058. Test r2: -0.152
Epoch [3/10]. Step [1/104]. Loss: 0.010. r2: -0.005. Test r2: -0.271
Epoch [3/10]. Step [31/104]. Loss: 0.009. r2: -1.759. Test r2: -0.191
Epoch [3/10]. Step [61/104]. Loss: 0.009. r2: -1.388. Test r2: -0.178
Epoch [3/10]. Step [91/104]. Loss: 0.009. r2: -1.284. Test r2: -0.182
Epoch [4/10]. Step [1/104]. Loss: 0.010. r2: -0.031. Test r2: -0.245
Epoch [4/10]. Step [31/104]. Loss: 0.009. r2: -1.826. Test r2: -0.177
Epoch [4/10]. Step [61/1

In [8]:
train_nn(opt='sgd', num_epochs=10)

Epoch [1/10]. Step [1/104]. Loss: 0.010. r2: 0.005. Test r2: -0.289
Epoch [1/10]. Step [31/104]. Loss: 0.009. r2: -0.331. Test r2: -0.265
Epoch [1/10]. Step [61/104]. Loss: 0.009. r2: -0.268. Test r2: -0.243
Epoch [1/10]. Step [91/104]. Loss: 0.009. r2: -0.240. Test r2: -0.242
Epoch [2/10]. Step [1/104]. Loss: 0.010. r2: 0.003. Test r2: -0.251
Epoch [2/10]. Step [31/104]. Loss: 0.009. r2: -0.308. Test r2: -0.240
Epoch [2/10]. Step [61/104]. Loss: 0.009. r2: -0.273. Test r2: -0.237
Epoch [2/10]. Step [91/104]. Loss: 0.009. r2: -0.220. Test r2: -0.223
Epoch [3/10]. Step [1/104]. Loss: 0.010. r2: -0.009. Test r2: -0.240
Epoch [3/10]. Step [31/104]. Loss: 0.009. r2: -0.181. Test r2: -0.233
Epoch [3/10]. Step [61/104]. Loss: 0.009. r2: -0.414. Test r2: -0.231
Epoch [3/10]. Step [91/104]. Loss: 0.009. r2: -0.306. Test r2: -0.228
Epoch [4/10]. Step [1/104]. Loss: 0.010. r2: 0.005. Test r2: -0.235
Epoch [4/10]. Step [31/104]. Loss: 0.009. r2: -0.241. Test r2: -0.229
Epoch [4/10]. Step [61/104]

Модель с оптимизатором Adam показала лучшие результаты.